In [38]:
# add genes to algoithms without genes

import os
import numpy as np
import pandas as pd
import sys,os
import random
import copy

import subprocess

import matplotlib.pyplot as plt
import seaborn as sns

from utils.eval import find_best_matches, generate_exprs

from methods import NMF, PCA, sparse_PCA, moCluster, MOFA2, iClusterPlus

gene_sets_are_defined = ['NMF', 'sparse_PCA']


classifications={"Intrinsic":["Luminal","Basal","Her2","Normal","Claudin-low"],
                "SCMOD2":["ER-/HER2-","ER+/HER2- Low Prolif","ER+/HER2- High Prolif","HER2+"],
                "IHC":["IHC_TNBC","IHC_ER","IHC_HER2","IHC_PR"]}

file_metabric_annotation = '/local/DESMOND2_data/v6/preprocessed_v6/METABRIC_1904.annotation_v6.tsv'
file_metabric_expression = '/local/DESMOND2_data/v6/preprocessed_v6/METABRIC_1904_17Kgenes.log2_exprs_z_v6.tsv'
file_metabric_subtypes = '/local/DESMOND2_data/v6/preprocessed_v6/METABRIC_1904_17Kgenes.subtypes_and_signatures_v6.tsv'
file_tcga_annotation = '/local/DESMOND2_data/v6/preprocessed_v6/TCGA-BRCA_1079.Xena_TCGA_PanCan.annotation_v6.tsv'
file_tcga_expression = '/local/DESMOND2_data/v6/preprocessed_v6/TCGA-BRCA_1079_17Kgenes.Xena_TCGA_PanCan.log2_exprs_z_v6.tsv'
file_tcga_subtypes = '/local/DESMOND2_data/v6/preprocessed_v6/TCGA-BRCA_1079_17Kgenes.Xena_TCGA_PanCan.subtypes_and_signatures_v6.tsv'
file_gene_mapping = '/local/DESMOND2_data/v6/preprocessed_v6/gene_id_mapping.tsv'

out_dir = '/cosybio/project/hartung/unpast/unpast_real'

basename_t = "TCGA"
basename_m = "METABRIC" 
result_m = None

def add_genes(METHOD):
    global result_m
    
    method_name = METHOD.__name__.split('.')[-1]
    print('method_name:', method_name)

    #### Preparation
    # METABRIC
    file_path_m = file_metabric_expression
    output_path_m = os.path.join(out_dir, basename_m, method_name)
    ground_truth_file_m = file_metabric_annotation
    combinations_m = METHOD.generate_arg_list(file_path_m, output_path_m, ground_truth_file_m)
    # TCGA
    file_path_t = file_tcga_expression
    output_path_t = os.path.join(out_dir, basename_t, method_name)
    ground_truth_file_t = file_tcga_annotation
    combinations_t = METHOD.generate_arg_list(file_path_t, output_path_t, ground_truth_file_t)
    
    for _iteration, (comb_m, comb_t) in enumerate(zip(combinations_m, combinations_t)):
        
        exprs_file = comb_m['exprs_file']
        result_m, _ = METHOD.run_real(comb_m, is_terminated=True)
        
        # save result as tsv for add genes input
        output_file = os.path.join(comb_m['output_path'], 'result.tsv')
        
        result_m['samples'] = result_m['samples'].map(list)
        result_m.to_csv(output_file, sep='\t', index=0)
        
        print('output_file', output_file)

        process_m = subprocess.Popen([f"Rscript", "/home/bba1401/Projects/unpast/DESMOND2/utils/add_genes.R", output_file, file_metabric_expression, "FALSE"])
        process_m.wait()
        
        
        
        result_t, _ = METHOD.run_real(comb_t, is_terminated=True)
        
        # save result as tsv for add genes input
        output_file = os.path.join(comb_t['output_path'], 'result.tsv')
        
        result_t['samples'] = result_t['samples'].map(list)
        
        result_t.to_csv(output_file, sep='\t', index=0)
        print('output_file', output_file)

        process_t = subprocess.Popen([f"Rscript", "/home/bba1401/Projects/unpast/DESMOND2/utils/add_genes.R", output_file, file_tcga_expression, "TRUE"])
        process_t.wait()
        

In [39]:
method = moCluster
add_genes(method)

method_name: moCluster
output_file /cosybio/project/hartung/unpast/unpast_real/METABRIC/moCluster/n_dimensions=1/n_cluster=2/random_state=1/solver=fast/center=True/method=globalScore/option=lambda1/scale=False/k=0.1/result.tsv
Coefficients not estimable: bic 


Error in contrasts.fit(fit, contrasts_matrix) : 
  trying to take contrast of non-estimable coefficient
Calls: apply -> FUN -> find_DE_genes -> contrasts.fit
In addition: Warning message:
Partial NA coefficients for 17158 probe(s) 
Execution halted


output_file /cosybio/project/hartung/unpast/unpast_real/TCGA/moCluster/n_dimensions=1/n_cluster=2/random_state=1/solver=fast/center=True/method=globalScore/option=lambda1/scale=False/k=0.1/result.tsv


Error: Negative counts not allowed
Execution halted


output_file /cosybio/project/hartung/unpast/unpast_real/METABRIC/moCluster/n_dimensions=1/n_cluster=2/random_state=1/solver=fast/center=True/method=globalScore/option=lambda1/scale=False/k=1/result.tsv
Coefficients not estimable: bic 


Error in contrasts.fit(fit, contrasts_matrix) : 
  trying to take contrast of non-estimable coefficient
Calls: apply -> FUN -> find_DE_genes -> contrasts.fit
In addition: Warning message:
Partial NA coefficients for 17158 probe(s) 
Execution halted


output_file /cosybio/project/hartung/unpast/unpast_real/TCGA/moCluster/n_dimensions=1/n_cluster=2/random_state=1/solver=fast/center=True/method=globalScore/option=lambda1/scale=False/k=1/result.tsv


In [ ]:
add_genes(iClusterPlus)

method_name: iClusterPlus
output_file /cosybio/project/hartung/unpast/unpast_real/METABRIC/iClusterPlus/random_state=1/n_cluster=2/type=gaussian/n_burnin=200/n_draw=200/lambda_n=5/maxiter=20/sdev=0.05/eps=0.0001/lambda_scale=1/result.tsv
Coefficients not estimable: bic 


Error in contrasts.fit(fit, contrasts_matrix) : 
  trying to take contrast of non-estimable coefficient
Calls: apply -> FUN -> find_DE_genes -> contrasts.fit
In addition: Warning message:
Partial NA coefficients for 17158 probe(s) 
Execution halted


output_file /cosybio/project/hartung/unpast/unpast_real/TCGA/iClusterPlus/random_state=1/n_cluster=2/type=gaussian/n_burnin=200/n_draw=200/lambda_n=5/maxiter=20/sdev=0.05/eps=0.0001/lambda_scale=1/result.tsv


Error: Negative counts not allowed
Execution halted


In [ ]:
add_genes(MOFA2)

method_name: MOFA2
output_file /cosybio/project/hartung/unpast/unpast_real/METABRIC/MOFA2/n_factors=1/n_cluster=2/random_state=1/ard_weights=True/ard_factors=False/likelihood=gaussian/spikeslab_weights=True/spikeslab_factors=False/result.tsv
Coefficients not estimable: bic 


Error in contrasts.fit(fit, contrasts_matrix) : 
  trying to take contrast of non-estimable coefficient
Calls: apply -> FUN -> find_DE_genes -> contrasts.fit
In addition: Warning message:
Partial NA coefficients for 17158 probe(s) 
Execution halted


output_file /cosybio/project/hartung/unpast/unpast_real/TCGA/MOFA2/n_factors=1/n_cluster=2/random_state=1/ard_weights=True/ard_factors=False/likelihood=gaussian/spikeslab_weights=True/spikeslab_factors=False/result.tsv


Error: Negative counts not allowed
Execution halted


In [ ]:
result_m['samples'].map(list)

1    [MB-7256, MB-4724, MB-7249, MB-0243, MB-0884, ...
2    [MB-5360, MB-6012, MB-4343, MB-5174, MB-6019, ...
Name: samples, dtype: object

In [37]:
pd.read_csv('/cosybio/project/hartung/unpast/unpast_real/METABRIC/moCluster/n_dimensions=1/n_cluster=2/random_state=1/solver=fast/center=True/method=globalScore/option=lambda1/scale=False/k=0.1/result.tsv', sep='\t')

,samples
0,"['MB-7256', 'MB-4724', 'MB-7249', 'MB-0243', '..."
1,"['MB-4046', 'MB-6012', 'MB-4343', 'MB-5174', '..."


Error: Negative counts not allowed
Execution halted


Coefficients not estimable: bic 


Error in contrasts.fit(fit, contrasts_matrix) : 
  trying to take contrast of non-estimable coefficient
Calls: apply -> FUN -> find_DE_genes -> contrasts.fit
In addition: Warning message:
Partial NA coefficients for 17158 probe(s) 
Execution halted


Coefficients not estimable: bic 


Error in contrasts.fit(fit, contrasts_matrix) : 
  trying to take contrast of non-estimable coefficient
Calls: apply -> FUN -> find_DE_genes -> contrasts.fit
In addition: Warning message:
Partial NA coefficients for 17158 probe(s) 
Execution halted


In [ ]:
pd.read_csv('/cosybio/project/hartung/unpast/unpast_real/TCGA/moCluster/n_dimensions=1/n_cluster=2/random_state=1/solver=fast/center=True/method=globalScore/option=lambda1/scale=False/k=0.1/result.tsv', sep='\t')

,samples,n_samples
0,"{'TCGA-B6-A1KC-01', 'TCGA-AC-A4ZE-01', 'TCGA-A...",983
1,"{'TCGA-GM-A5PX-01', 'TCGA-A7-A425-01', 'TCGA-U...",96
